<a href="https://colab.research.google.com/github/callumeppz/C-Revision/blob/main/Copy_of_CW_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prior study (Codes in Week 5 folder）**:

Please use this page as companion to understand the newsgroup data set.
[Data Set](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

You will also need to be familiar with some text processing commands：

[Tf-idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)

[countvectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)


In [3]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

# **Steps outline**
1. Download your data set by inputting your student number.
2. Process your text data, extract features, convert them into vectors
3. Modeling, train models on the data set (select model, tune different parameters)
4. Process your text data, extract features, convert them into vectors
5. Analysis and discussions

# Step 1: Load Dataset


Prior study (Codes in Week 5 folder）:

Please use this page as companion to understand  [**Newsgroup data set**](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',  categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',  categories=categories, shuffle=True, random_state=42)

**This is how to identify which data set to use (Please copy  the following information in report front   page).**

In [5]:
index=input('type your student number?')

type your student number?30010641


In [6]:
x=divmod(int(index),4)
yourdata1=x[1]
y=divmod(int(index),3)
yourdata2=y[1]

print('This is your data set index ----> (', x[1], y[1], ')' )

This is your data set index ----> ( 1 0 )


In [7]:
data1= twenty_train.target_names[x[1]]
data2= twenty_train.target_names[y[1]]
categories1=[data1,data2]
print(categories1)

['comp.graphics', 'alt.atheism']


**Your front page data information Ends here**

# Step 2 Process your text data, extract features

# 2.1 An example of preprocessing

**An example is provided.**

Please pay attention  comment #replace ..., which means you need to change example text to your data set.
Use google search for usages of  "nltk tokenizer ”, "nltk stemmer", "nltk pos tag" to help your report writing.

In [8]:
# write your own NLP precessing examples with  preprocessing techniques.

dataset=twenty_train.data[1]
print(dataset)
# please   replace 1 in bracket to other data sample and explore the code

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


From: ani@ms.uky.edu (Aniruddha B. Deglurkar)
Subject: help: Splitting a trimming region along a mesh 
Organization: University Of Kentucky, Dept. of Math Sciences
Lines: 28



	Hi,

	I have a problem, I hope some of the 'gurus' can help me solve.

	Background of the problem:
	I have a rectangular mesh in the uv domain, i.e  the mesh is a 
	mapping of a 3d Bezier patch into 2d. The area in this domain
	which is inside a trimming loop had to be rendered. The trimming
	loop is a set of 2d Bezier curve segments.
	For the sake of notation: the mesh is made up of cells.

	My problem is this :
	The trimming area has to be split up into individual smaller
	cells bounded by the trimming curve segments. If a cell
	is wholly inside the area...then it is output as a whole ,
	else it is trivially rejected. 

	Does any body know how thiss can be done, or is there any algo. 
	somewhere for doing this.

	Any help would be appreciated.

	Thanks, 
	Ani.
-- 
To get irritated is human, to stay cool, divi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:

# tokenize: search: nltk tokenize
example = "This is an example sentence."

from nltk.tokenize import word_tokenize
example_tokenize =word_tokenize(example)
#example_tokenize= word_tokenize(dataset) # replace example in bracket to dataset.
print("-------------------------tokenize:")
print(example_tokenize)

-------------------------tokenize:
['This', 'is', 'an', 'example', 'sentence', '.']


In [10]:
# stemmer: search: nltk stemmer
stemmer = nltk.stem.PorterStemmer()
example_stem = stemmer.stem(example)  # replace .....
print("-------------------------stem:")
print(example_stem)

-------------------------stem:
this is an example sentence.


In [11]:
# pos_taging: search: nltk pos tagging example
example_posTag=nltk.pos_tag(example_tokenize)
print("-------------------------pos_taging:")
print(example_posTag)

-------------------------pos_taging:
[('This', 'DT'), ('is', 'VBZ'), ('an', 'DT'), ('example', 'NN'), ('sentence', 'NN'), ('.', '.')]


In [12]:
 # consituency parsing, chunking
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(example_posTag)
print(result)

(S This/DT is/VBZ (NP an/DT example/NN) (NP sentence/NN) ./.)


#2.2 NLP Preprocesssing

**Some preprocessing are provided for convenience. Please include why NLP preprocessing is in your report. Explain what techniques have been experimented in your report.**

In [13]:
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from tqdm import tqdm
from nltk.corpus import stopwords
stopwordEn = stopwords.words('english')
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

def lemmaWord(word):
    lemma = wordnet.morphy(word)
    if lemma is not None:
        return lemma
    else:
        return word

def stemWord(word):
    stem = stemmer.stem(word)
    if stem is not None:
        return stem
    else:
        return word

def processText(text,lemma=False, gram=1, rmStop=True): # default remove stop words
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b|@\w+|#', '', text, flags=re.MULTILINE) #delete URL, #hashtag# , and @xxx
    tokens = word_tokenize(text)
    whitelist = ["n't", "not", "no"]
    new_tokens = []
    stoplist = stopwordEn if rmStop else []
    for i in tokens:
      i = i.lower()
      if i.isalpha() and (i not in stoplist or i in whitelist):  #i not in ['.',',',';']  and (...)
        if lemma: i = lemmaWord(i)
        new_tokens.append(i)
    del tokens
    # tokens = [lemmaWord(i.lower()) if lemma else i.lower() for i in tokens if (i.lower() not in stoplist or i.lower() in whitelist) and i.isalpha()]
    if gram<=1:
        return new_tokens
    else:
        return [' '.join(i) for i in nltk.ngrams(new_tokens, gram)]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def getTags(text):
  token = word_tokenize(text)
  token = [l.lower() for l in token]
  train_tags = nltk.pos_tag(token)
  return [i[1] for i in train_tags]


In [15]:
print(processText(dataset))

['aniruddha', 'deglurkar', 'subject', 'help', 'splitting', 'trimming', 'region', 'along', 'mesh', 'organization', 'university', 'kentucky', 'dept', 'math', 'sciences', 'lines', 'hi', 'problem', 'hope', 'help', 'solve', 'background', 'problem', 'rectangular', 'mesh', 'uv', 'domain', 'mesh', 'mapping', 'bezier', 'patch', 'area', 'domain', 'inside', 'trimming', 'loop', 'rendered', 'trimming', 'loop', 'set', 'bezier', 'curve', 'segments', 'sake', 'notation', 'mesh', 'made', 'cells', 'problem', 'trimming', 'area', 'split', 'individual', 'smaller', 'cells', 'bounded', 'trimming', 'curve', 'segments', 'cell', 'wholly', 'inside', 'area', 'output', 'whole', 'else', 'trivially', 'rejected', 'body', 'know', 'thiss', 'done', 'algo', 'somewhere', 'help', 'would', 'appreciated', 'thanks', 'ani', 'get', 'irritated', 'human', 'stay', 'cool', 'divine']


In [16]:
print(getTags(dataset))

['IN', ':', 'NN', 'NN', 'NN', '(', 'JJ', 'NN', 'NN', ')', 'NN', ':', 'NN', ':', 'VBG', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', ':', 'NN', 'IN', 'NN', ',', 'NN', '.', 'IN', 'NN', 'NNS', 'NNS', ':', 'CD', 'NN', ',', 'NN', 'VBP', 'DT', 'NN', ',', 'NN', 'VBP', 'DT', 'IN', 'DT', 'NNP', 'POS', 'MD', 'VB', 'PRP', 'VB', '.', 'NN', 'IN', 'DT', 'NN', ':', 'NN', 'VBP', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', ',', 'VBP', 'DT', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'CD', 'NN', 'NN', 'IN', 'CD', '.', 'DT', 'NN', 'IN', 'DT', 'NN', 'WDT', 'VBZ', 'IN', 'DT', 'NN', 'NN', 'VBD', 'TO', 'VB', 'VBN', '.', 'DT', 'VBG', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'CD', 'NN', 'NN', 'NNS', '.', 'IN', 'DT', 'NN', 'IN', 'NN', ':', 'DT', 'NN', 'VBZ', 'VBN', 'IN', 'IN', 'NNS', '.', 'PRP$', 'NN', 'VBZ', 'DT', ':', 'DT', 'VBG', 'NN', 'VBZ', 'TO', 'VB', 'VBN', 'RP', 'IN', 'JJ', 'JJR', 'NNS', 'VBN', 'IN', 'DT', 'VBG', 'NN', 'NNS', '.', 'IN', 'DT', 'NN', 'VBZ', 'RB', 'IN', 'DT', 'NN', ':', 'RB', 'PRP', 'VBZ', 'NN', 'IN', 'DT', 'N

# Step 3: Build a Pipeline

**Modify the block code below to your choice of classifier [link text](https://www.nltk.org/book/ch06.html)

In [17]:
print(categories1)


['comp.graphics', 'alt.atheism']


Without modification, the code will output all four classes.


I included some commented codes in places where you may use to change to two class data sets   from your student number, and use logistic model.
Your data sets can be obtained as twenty_train1, twenty_test1. All  data set names can be adjusted to get it right.

In [18]:
twenty_train1 = fetch_20newsgroups(subset='train',  categories=categories1, shuffle=True, random_state=42)
twenty_test1 = fetch_20newsgroups(subset='test',  categories=categories1, shuffle=True, random_state=42)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression

# Level: lexicon, model: tf-idf
text_clf = Pipeline([
    # add your code about text processing
    ('vect', CountVectorizer(analyzer=processText)),
    ('tfidf', TfidfTransformer(use_idf=True)),

    # change your classifier here, search: sklearn logistic regression example
    #('clf', SGDClassifier())
    ('clf', LogisticRegression())

])

In [20]:
# To train the model
text_clf.fit(twenty_train1.data, twenty_train1.target)


Pipeline(steps=[('vect',
                 CountVectorizer(analyzer=<function processText at 0x7b90d8be4310>)),
                ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())])

# Step 4: Make Prediction

In [21]:
# To make prediction with dev/test set
predicted = text_clf.predict(twenty_test1.data)

# Step 5: Evaluation

**You need to modify the code so only two classes from your student number are output as matrix.**

In [22]:
# To evaluate your prediction on dev set
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(twenty_test1.target, predicted))

print(metrics.classification_report(twenty_test1.target, predicted, target_names=twenty_test1.target_names))

# confusion class
pd.DataFrame(metrics.confusion_matrix(twenty_test1.target, predicted),
             columns=twenty_test1.target_names,index=twenty_test1.target_names)

Accuracy: 0.9505649717514124
               precision    recall  f1-score   support

  alt.atheism       0.98      0.91      0.94       319
comp.graphics       0.93      0.99      0.96       389

     accuracy                           0.95       708
    macro avg       0.96      0.95      0.95       708
 weighted avg       0.95      0.95      0.95       708



,alt.atheism,comp.graphics
alt.atheism,289,30
comp.graphics,5,384


# Step 6: Error Analysis and Discussion
write down your own obseration about the predictions. Consider both confusion matrix and selected examples. Which classes are predicted correctly or incorrecly, possible explaination, possible solutions

Exmaple: 1) Lab Practical, which feature is helpful for female name classification. https://www.nltk.org/book/ch06.html
2) research paper: https://github.com/yoonkim/CNN_sentence


In [23]:
df_pred = pd.DataFrame({'news':twenty_test1.data,'prediction':predicted, 'true':twenty_test1.target})
df_pred[df_pred['true'] != df_pred['prediction']]

,news,prediction,true
0,From: aaron@minster.york.ac.uk\nSubject: Re: G...,1,0
4,Organization: Penn State University\nFrom: <SM...,1,0
15,From: jk87377@lehtori.cc.tut.fi (Kouhia Juhana...,0,1
20,From: kax@cs.nott.ac.uk (Kevin Anthoney)\nSubj...,1,0
82,From: aaron@minster.york.ac.uk\nSubject: Re: D...,1,0
95,From: johnchad@triton.unm.edu (jchadwic)\nSubj...,1,0
109,From: gck@aero.org (Gregory C. Kozlowski)\nSub...,1,0
123,From: pww@spacsun.rice.edu (Peter Walker)\nSub...,1,0
145,From: pstlb@aurora.alaska.edu\nSubject: Where ...,0,1
173,From: val@fcom.cc.utah.edu (Val Kartchner)\nSu...,0,1


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

#tfidf_vectorizer = TfidfVectorizer(max_features=10000)
#X_tfidf = tfidf_vectorizer.fit_transform(X)

#X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

#classifier = MultinomialNB()
#classifier.fit(X_train, y_train)

#predicted_labels = classifier.predict(X_test)

In [29]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

predicted_labels = df_pred['prediction']
true_labels = df_pred['true']

accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)

print('Accuracy:', accuracy)
print('F1 score:', f1)
print('Recall:', recall)
print('Precision:', precision)


Accuracy: 0.9505649717514124
F1 score: 0.9503264530804204
Recall: 0.987146529562982
Precision: 0.927536231884058


#References:  

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

https://www.nltk.org/book/ch06.html

search: Other online resources:

https://towardsdatascience.com/setting-up-text-preprocessing-pipeline-using-scikit-learn-and-spacy-e09b9b76758f

sentiment analysis scikit learn

scikit learn or nltk + NLP techniques

python + NLP techniques

scikit learn logistic regression


